In [12]:
from florence_utils import run_inference
from llama_utils import call_llama,call_llama_combine
from ram_utils import load_tags_from_json,get_tags_for_id
from paddle_utililty import extract_text_from_image

cuda:0


c:\Users\redti\anaconda3\envs\cvproject\Lib\site-packages\fairscale\experimental\nn\offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
c:\Users\redti\anaconda3\envs\cvproject\Lib\site-packages\fairscale\experimental\nn\offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


[2024/12/19 00:45:19] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\redti/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\redti/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

In [13]:
# from transformers import OwlViTProcessor, OwlViTForObjectDetection
# from PIL import Image
# import requests
# import torch

# # Device setup
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Load the model and processor
# model_name = "google/owlvit-base-patch16"
# processor = OwlViTProcessor.from_pretrained(model_name)
# model = OwlViTForObjectDetection.from_pretrained(model_name).to(device).eval()

# # Load an example image 



In [14]:
# image = Image.open("data/image/AMBER_1.jpg").convert("RGB")

# # Define text queries
# text_queries = [
#             "couple",
#             "field",
#             "hand",
#             "grass",
#             "grassy",
#             "green",
#             "hill",
#             "hillside",
#             "person",
#             "lush",
#             "man",
#             "pasture",
#             "walk",
#             "woman"
#         ]

# # Process inputs
# inputs = processor(text=text_queries, images=image, return_tensors="pt").to(device)

# # Inference
# with torch.no_grad():
#     outputs = model(**inputs)

# # Post-processing
# target_sizes = torch.tensor([image.size[::-1]])  # (height, width)
# results = processor.post_process(outputs=outputs, target_sizes=target_sizes)
# result = results[0]


# scores = result["scores"]
# labels = result["labels"]
# boxes = result["boxes"]


# threshold = 0.2
# high_conf_indices = scores > threshold
# filtered_boxes = boxes[high_conf_indices]
# filtered_scores = scores[high_conf_indices]
# filtered_labels = labels[high_conf_indices]

# # Display detected objects
# for label, box, score in zip(filtered_labels, filtered_boxes, filtered_scores):
#     print(f"Detected '{text_queries[label]}' with confidence {score.item():.2f}")

In [15]:
import os
import json
import shutil
from PIL import Image

# Directory containing the images
image_directory = "data/image"
tags_json_file = "data/tags.json"
tags_dict = load_tags_from_json(tags_json_file)
output_data = []

# Create a temporary directory name
temp_dir = "temp_slices"

for i in range(1, 1005):
    try:
        img_path = f"{image_directory}/AMBER_{i}.jpg"

        # Open the image
        with Image.open(img_path) as img:
            width, height = img.size

            # Compute mid points (if not evenly divisible, the right/bottom gets the extra pixel)
            mid_w = width // 2
            mid_h = height // 2

            # Coordinates for cropping into 4 pieces
            # Top-left
            box1 = (0, 0, mid_w, mid_h)
            # Top-right
            box2 = (mid_w, 0, width, mid_h)
            # Bottom-left
            box3 = (0, mid_h, mid_w, height)
            # Bottom-right
            box4 = (mid_w, mid_h, width, height)

            # Ensure temp directory is clean
            if os.path.exists(temp_dir):
                shutil.rmtree(temp_dir)
            os.makedirs(temp_dir, exist_ok=True)

            # Crop and save the four pieces
            piece_paths = []
            for idx, box in enumerate([box1, box2, box3, box4], start=1):
                cropped_img = img.crop(box)
                cropped_path = os.path.join(temp_dir, f"slice_{idx}.jpg")
                cropped_img.save(cropped_path)
                piece_paths.append(cropped_path)

        # Generate captions for each piece
        caption_list = []
        for piece_path in piece_paths:
            piece_caption = run_inference(piece_path, '<CAPTION>')
            caption_list.append(piece_caption)

        # Cleanup temp directory
        shutil.rmtree(temp_dir)

        # Combine captions using your llama function
        final_response = call_llama_combine(caption_list)
        
        print(f"Response for image {i}: {final_response}")
        output_data.append({"id": i, "response": final_response})

    except Exception as e:
        print(f"Issue on ID: {i}, Error: {e}")

# Save the results to a JSON file
output_file = "evaluation_results/amber_evaluation_results.json"
with open(output_file, "w") as file:
    json.dump(output_data, file, indent=4)

print(f"Evaluation results saved to {output_file}")


Response for image 1: A serene landscape unfolds, with a woman walking through a lush green field towards a distant group of people, set against a breathtaking backdrop of mountains visible from the top of a hill.
Response for image 2: A man is peacefully floating on a canoe in the calm water, surrounded by lush green trees as the sun shines bright in the sky with a vibrant rainbow visible above.
Response for image 3: A family is enjoying a beautiful day outdoors, with children playing on a lush green field and lawn while one of them throws a frisbee, surrounded by vibrant yellow flowers scattered throughout the grass.
Response for image 4: A woman enjoying a serene day on the lake, surrounded by lush green fields and large trees.
Response for image 5: A serene beach scene unfolds, with a calm body of water stretching towards the horizon where a boat glides effortlessly. In the distance, children's laughter echoes as a little girl, clad in a bathing suit, excitedly runs into the waves,

In [16]:
import os
import json

# Directory containing the images
image_directory = "data/image"
tags_json_file = "data/tags.json"
tags_dict = load_tags_from_json(tags_json_file)
output_data = []

# Loop through each image file and perform the inference for each task
for i in range(1, 1005):
    try:
    # Construct the image path
        img_path =  f"{image_directory}/AMBER_{i}.jpg"

        # Run inference with each task type
        # image_tags = get_tags_for_id(i, tags_dict)
        # object_detection_info = run_inference(img_path, '<OD>')
        # ocr_info = extract_text_from_image(img_path)
        caption = run_inference(img_path, '<CAPTION>')
        
        caption_list=

        final_response=call_llama_combine(caption_list)
        # Generate final response using call_llama
        # base64_image=convert_to_base64(img_path)
        # final_response = call_llama(ocr_info, object_detection_info,base64_image)
        # final_response = call_llama(ocr_info, object_detection_info,image_tags,caption)
        
        print(f"Response for image {i}: {final_response}")
        # print(f"Basic Response for image {i}: {basic_response}")
        # Append the result for this image
        output_data.append({"id": i, "response": final_response})
    except:
        print("Issue on ID: "+str(i))

# Save the results to a JSON file
output_file = "evaluation_results/amber_evaluation_results.json"
with open(output_file, "w") as file:
    json.dump(output_data, file, indent=4)

print(f"Evaluation results saved to {output_file}")
